In [56]:
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.


## Data Import

### We need to use Kaggle's API to download the data since it's too big to be uploaded to GitHub

In [57]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("usdot/flight-delays")

print("Path to dataset files:", path)

Path to dataset files: /Users/naominayman/.cache/kagglehub/datasets/usdot/flight-delays/versions/1


In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [59]:
import tensorflow as tf
from tensorflow import keras
# could be cool to do sometype of simple NN 

In [60]:
df = pd.read_csv(path + "/flights.csv")

/var/folders/78/46rnh6cd7w1dzd2t3dyd5t040000gn/T/ipykernel_13945/2301439082.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "/flights.csv")


In [61]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
# for day of the week, 1 is Monday, 7 is Sunday
# we don't want any model to mistake the day of the week as numeric data
df['DAY_OF_WEEK'] = df['DAY_OF_WEEK'].map({ 1: 'M',
                                            2: 'Tu',
                                            3: 'W',
                                            4: 'Th',
                                            5: 'F',
                                            6: 'Sa',
                                            7: 'Su'})

In [63]:
df.DEPARTURE_DELAY = df.DEPARTURE_DELAY.dropna() # drop NaN
df.ARRIVAL_DELAY = df.ARRIVAL_DELAY.dropna() # drop NaN

### We need to decide how long the departure/arrival delay needs to be to classify as a significant delay

This is just my data exploration

In [64]:
df.DEPARTURE_DELAY.describe(), df.ARRIVAL_DELAY.describe()

(count    5.732926e+06
 mean     9.370158e+00
 std      3.708094e+01
 min     -8.200000e+01
 25%     -5.000000e+00
 50%     -2.000000e+00
 75%      7.000000e+00
 max      1.988000e+03
 Name: DEPARTURE_DELAY, dtype: float64,
 count    5.714008e+06
 mean     4.407057e+00
 std      3.927130e+01
 min     -8.700000e+01
 25%     -1.300000e+01
 50%     -5.000000e+00
 75%      8.000000e+00
 max      1.971000e+03
 Name: ARRIVAL_DELAY, dtype: float64)

In [65]:
df.DEPARTURE_DELAY.value_counts().sort_index()

DEPARTURE_DELAY
-82.0      1
-68.0      1
-61.0      1
-56.0      1
-55.0      1
          ..
 1631.0    1
 1649.0    1
 1670.0    1
 1878.0    1
 1988.0    1
Name: count, Length: 1217, dtype: int64

In [66]:
df.ARRIVAL_DELAY.value_counts().sort_index()

ARRIVAL_DELAY
-87.0      2
-82.0      1
-81.0      2
-80.0      3
-79.0      2
          ..
 1636.0    2
 1638.0    1
 1665.0    1
 1898.0    1
 1971.0    1
Name: count, Length: 1240, dtype: int64

In [67]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [ ]:
# We need to do some data exploration for the columns that we want to use
# we'll definitely need to drop some columns and rows
# we'll also need to figure out which airlines we want to keep because it'd be too much to encode all of them
# we'll also need to figure out which airports we want to keep because it'd be too much to encode all of them